<a href="https://colab.research.google.com/github/bytehub-ai/code-examples/blob/main/bytehub_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q bytehub

     |████████████████████████████████| 122kB 7.5MB/s 
     |████████████████████████████████| 17.7MB 198kB/s 
     |████████████████████████████████| 51kB 6.8MB/s 


In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# **ByteHub Feature Store Demo**

**Import** ByteHub

In [3]:
import bytehub as bh
bh.__version__

'0.0.7'

**Load** and **login** to **Feature Store**

In [4]:
fs = bh.FeatureStore(
    endpoint='https://api.dev.bytehub.ai/v1',
    client_id='rlnnlteca0oq01bk081kivg4s'
)

Please go to https://bytehub-ai-dev.auth.eu-west-2.amazoncognito.com/login?response_type=code&client_id=rlnnlteca0oq01bk081kivg4s&redirect_uri=https%3A%2F%2Fwww.bytehub.ai%2Fauthenticated&state=6h2WMBRGdSezZA4rtjzGPCcPvIo2OD and login. Copy the response code and paste below.
Response: ··········


## **Data Access**

ByteHub Feature Stores are **preloaded** with **prepared production-ready data**, e.g **hyperlocal weather data**

In [5]:
fs.list_features()[:5]

['noaa.data.gfs-uk',
 'bmrs.feature.total-wind',
 'test.my-new-feature',
 'carbonintensity.data.intensity',
 'carbonintensity.feature.actual']

Features are stored with **metadata**

In [6]:
fs.list_features(metadata=True).head(5)

,name,metadata
0,noaa.data.gfs-uk,{'source': 'https://www.ncdc.noaa.gov/data-acc...
1,bmrs.feature.total-wind,{}
2,test.my-new-feature,"{'animal': 'dog', 'source': 'demo'}"
3,carbonintensity.data.intensity,{'source': 'https://api.carbonintensity.org.uk/'}
4,carbonintensity.feature.actual,{'source': 'https://api.carbonintensity.org.uk/'}


## **Data Discovery**

Features are **regex searchable** 

In [7]:
# Search for the BMRS features
fs.list_features(regex=r'bmrs\..')[:5]

['bmrs.feature.total-wind',
 'bmrs.data.bod',
 'bmrs.feature.lowest-offer',
 'bmrs.feature.highest-bid',
 'bmrs.data.indoitsdo']

## **Optimised for timeseries**

Lets **query** and **aggregate** some features


In [8]:
df = fs.get_timeseries(['bmrs.feature.system-price', 'bmrs.feature.niv'], from_date='2020-09-01', to_date=pd.Timestamp.utcnow(), freq='30min')
df.head()

,time,entity,bmrs.feature.system-price,bmrs.feature.niv
0,2020-09-01 00:00:00+00:00,None,46.4,249.9333
1,2020-09-01 00:30:00+00:00,None,46.3,335.4755
2,2020-09-01 01:00:00+00:00,None,46.3,437.5139
3,2020-09-01 01:30:00+00:00,None,46.0,239.6694
4,2020-09-01 02:00:00+00:00,None,46.0,316.8444


Lets **plot** the features in a **chart**

In [9]:
traces = [
          go.Scatter(x=df.time, y=df['bmrs.feature.system-price'], name='System Price'),
          go.Scatter(x=df.time, y=df['bmrs.feature.niv'], name='NIV', yaxis='y2'),
]
layout = {
    'title': 'System Prices and NIV',
    'template': 'seaborn',
    'yaxis': {'title': 'System price'},
    'yaxis2': {'title': 'Net Imbalance Volume', 'overlaying': 'y', 'side': 'right'},
}
fig = go.Figure(data=traces, layout=layout)
fig

We can **upsample**

In [10]:
fs.get_timeseries(['bmrs.feature.system-price', 'bmrs.feature.niv'], from_date='2020-09-01', to_date=pd.Timestamp.utcnow(), freq='10min').head()

,time,entity,bmrs.feature.system-price,bmrs.feature.niv
0,2020-09-01 00:00:00+00:00,None,46.4,249.9333
1,2020-09-01 00:10:00+00:00,None,46.4,249.9333
2,2020-09-01 00:20:00+00:00,None,46.4,249.9333
3,2020-09-01 00:30:00+00:00,None,46.3,335.4755
4,2020-09-01 00:40:00+00:00,None,46.3,335.4755


Or **downsample**

In [11]:
fs.get_timeseries(['bmrs.feature.system-price', 'bmrs.feature.niv'], from_date='2020-09-01', to_date=pd.Timestamp.utcnow(), freq='1d').head()

,time,entity,bmrs.feature.system-price,bmrs.feature.niv
0,2020-09-01 00:00:00+00:00,None,46.4,249.9333
1,2020-09-02 00:00:00+00:00,None,15.0,-4.6359
2,2020-09-03 00:00:00+00:00,None,51.5,231.7499
3,2020-09-04 00:00:00+00:00,None,2.0,-401.1833
4,2020-09-05 00:00:00+00:00,None,2.0,-128.9746


For **raw features** with **different frequencies**



In [12]:
fs.get_freq('bmrs.feature.rolling-demand')

'5T'

In [13]:
fs.get_freq('bmrs.feature.system-price')

'30T'

**`get_timeseries`** will **resample** and **merge** timeseries at a **consistent frequency**

In [14]:
fs.get_timeseries(['bmrs.feature.rolling-demand', 'bmrs.feature.system-price'], from_date='2020-09-01', to_date=pd.Timestamp.utcnow(), freq='30min').head()

,time,entity,bmrs.feature.rolling-demand,bmrs.feature.system-price
0,2020-09-01 00:00:00+00:00,None,21456.0,46.4
1,2020-09-01 00:30:00+00:00,None,21488.0,46.3
2,2020-09-01 01:00:00+00:00,None,21252.0,46.3
3,2020-09-01 01:30:00+00:00,None,21293.0,46.0
4,2020-09-01 02:00:00+00:00,None,20967.0,46.0


We can also use **`get_timeseries`** to **time_travel** by returning features that were available at a specified point in time. For example, return the weather forecasts that were available at least 6 hours in advance.

In [16]:
fs.get_timeseries('noaa.data.gfs-uk', from_date='2020-11-01', to_date='2020-11-10', freq='60min', time_travel='-6h').head()

,time,entity,noaa.data.gfs-uk
0,2020-11-01 00:00:00+00:00,None,"[{'GUST_surface': 11.1313114166, 'PRES_surface..."
1,2020-11-01 01:00:00+00:00,None,"[{'GUST_surface': 11.7034912109, 'PRES_surface..."
2,2020-11-01 02:00:00+00:00,None,"[{'GUST_surface': 13.0000200272, 'PRES_surface..."
3,2020-11-01 03:00:00+00:00,None,"[{'GUST_surface': 12.8234596252, 'PRES_surface..."
4,2020-11-01 04:00:00+00:00,None,"[{'GUST_surface': 13.3250379562, 'PRES_surface..."


## **Complex features**


In [ ]:
df = fs.get_timeseries('bmrs.data.dersysdata', from_date='2020-10-01', to_date='2020-10-03', freq='30min')
df.tail()

,time,entity,bmrs.data.dersysdata
92,2020-10-02 22:00:00+00:00,None,"[{'time': 1601676000000, 'activeFlag': 'Y', 'r..."
93,2020-10-02 22:30:00+00:00,None,"[{'time': 1601677800000, 'activeFlag': 'Y', 'r..."
94,2020-10-02 23:00:00+00:00,None,"[{'time': 1601679600000, 'activeFlag': 'Y', 'r..."
95,2020-10-02 23:30:00+00:00,None,"[{'time': 1601681400000, 'activeFlag': 'Y', 'r..."
96,2020-10-03 00:00:00+00:00,None,"[{'time': 1601683200000, 'activeFlag': 'Y', 'r..."


In [ ]:
df.iloc[-1, -1]

[{'activeFlag': 'Y',
  'bSADDefault': 'F',
  'buyPriceAdjustment': 0,
  'indicativeNetImbalanceVolume': 16.363,
  'priceDerivationCode': 'P',
  'recordType': 'SSB',
  'reserveScarcityPrice': 'NULL',
  'sellPriceAdjustment': 0,
  'settlementDate': 1601683200000,
  'settlementPeriod': 3,
  'systemBuyPrice': 50.0,
  'systemSellPrice': 50.0,
  'time': 1601683200000,
  'totalSystemAcceptedBidVolume': -368.035,
  'totalSystemAcceptedOfferVolume': 908.898,
  'totalSystemAdjustmentBuyVolume': 437.0,
  'totalSystemAdjustmentSellVolume': -961.5,
  'totalSystemTaggedAcceptedBidVolume': -368.035,
  'totalSystemTaggedAcceptedOfferVolume': 907.898,
  'totalSystemTaggedAdjustmentBuyVolume': 437.0,
  'totalSystemTaggedAdjustmentSellVolume': -961.5}]

In [ ]:
fs.get_last('bmrs.data.dersysdata')

,time,entity,value
0,2020-10-28 23:30:00+00:00,None,"[{'time': 1603927800000, 'activeFlag': 'Y', 'r..."


In [ ]:
fs.get_last('bmrs.data.rolsysdem').value[0]

[{'activeFlag': 'Y',
  'fuelTypeGeneration': 24518,
  'publishingPeriodCommencingTime': '2020-10-29 00:00:00',
  'recordType': 'VD',
  'settDate': '2020-10-29',
  'time': 1603929600000}]

In [ ]:
fs.get_last('bmrs.feature.demand-30min-history')

,time,entity,value
0,2020-10-29 00:00:00+00:00,None,"[24658, 24558, 24555, 24501, 24493, 24518]"


## **Develop Reusable Features**

**Create** a **new** feature

In [ ]:
fs.create_feature('test.my-new-feature', source='demo', animal='dog')

In [ ]:
dts = pd.date_range('2020-10-01', '2020-10-04', freq='1h')
df = pd.DataFrame({'time': dts, 'value': np.random.randint(0, 100, len(dts))})
fs.save_timeseries('test.my-new-feature', df)

In [ ]:
fs.get_timeseries('test.my-new-feature', from_date='2020-09-25', to_date='2020-10-04', freq='1d').head(5)

,time,entity,test.my-new-feature
0,2020-09-25 00:00:00+00:00,None,NaN
1,2020-09-26 00:00:00+00:00,None,NaN
2,2020-09-27 00:00:00+00:00,None,NaN
3,2020-09-28 00:00:00+00:00,None,NaN
4,2020-09-29 00:00:00+00:00,None,NaN


In [ ]:
fs.delete_feature('test.my-new-feature')

Build **new** features out of **old** - start by defining a **transform**.

In [ ]:
fuel_mix = fs.get_last('bmrs.data.fuelhh')
fuel_mix.iloc[0,-1]

{'activeFlag': 'Y',
 'biomass': 1858,
 'ccgt': 3422,
 'coal': 0,
 'intew': 0,
 'intfr': 1510,
 'intirl': 0,
 'intned': 158,
 'intnem': 382,
 'npshyd': 512,
 'nuclear': 6309,
 'ocgt': 0,
 'oil': 0,
 'other': 132,
 'ps': 0,
 'recordType': 'FUELHH',
 'settlementPeriod': 48,
 'startTimeOfHalfHrPeriod': 1603843200000,
 'wind': 10268}

In [ ]:
def total_renewable(x):
  return x.get('biomass', 0) + x.get('npshyd', 0) + x.get('wind', 0)

fs.create_transform('total-renewables', func=total_renewable)
fs.list_transforms()

['identity',
 'double',
 'extract-key',
 'sum-interconnectors',
 'total-renewables']

In [ ]:
fs.create_virtual_feature('bmrs.feature.total-renewables', 'total-renewables', derived_from='bmrs.data.fuelhh')

Now get a timeseries of this new, **virtual feature**


In [ ]:
fs.get_timeseries('bmrs.feature.total-renewables', from_date='2020-10-01', to_date='2020-10-02', freq='30min').head()

,time,entity,bmrs.feature.total-renewables
0,2020-10-01 00:00:00+00:00,None,8747
1,2020-10-01 00:30:00+00:00,None,8545
2,2020-10-01 01:00:00+00:00,None,8282
3,2020-10-01 01:30:00+00:00,None,8004
4,2020-10-01 02:00:00+00:00,None,7574


In [ ]:
fs.delete_feature('bmrs.feature.total-renewables')
fs.delete_transform('total-renewables')